In [1]:
import pandas as pd
import requests

In [3]:
# Get scriptures in json
json = requests.get('https://raw.githubusercontent.com/beandog/lds-scriptures/master/json/lds-scriptures-json.txt').json()

In [6]:
# Create a dataframe
df = pd.DataFrame(json)

In [28]:
# Save dataframe to parquet
df.to_parquet('lds-scriptures.parquet')

In [1]:
import polars as pl
import spacy 
nlp = spacy.load('en_core_web_md')

In [16]:
# Read parquet file
scriptures = pl.read_parquet("./data/scriptures.parquet")
# scriptures = pl.read_parquet("./data/scriptures.parquet")\
    # .head(1)\
    # .with_columns(
    #     pl.col("scripture_text").map_elements(lambda x: nlp(x)).alias("vec")
    # )


In [24]:
import spacy
from spacy.tokens import DocBin
nlp = spacy.load("en_core_web_md")
doc_bin = DocBin()

verses = scriptures.select("scripture_text").to_series().to_list()
verses = list(nlp.pipe(verses))
for i in range(len(verses)):
    verses[i] = ' '.join([token.text for token in verses[i] if not token.is_stop and not token.is_punct])

verses = nlp.pipe(verses)

for doc in verses:
    doc_bin.add(doc)
bytes_data = doc_bin.to_bytes()

In [25]:
# Save the bytes data somewhere
with open("scriptures2.spacy", "wb") as f:
    f.write(bytes_data)

# # Deserialize later, e.g. in a new process
# nlp = spacy.blank("en")
# doc_bin = DocBin().from_bytes(bytes_data)
# docs = list(doc_bin.get_docs(nlp.vocab))

In [26]:
# Deserialize later, e.g. in a new process
bytes_data = open("scriptures2.spacy", "rb").read()
nlp = spacy.load("en_core_web_md")
doc_bin = DocBin().from_bytes(bytes_data)
docs = list(doc_bin.get_docs(nlp.vocab))

In [27]:
most_similar(docs[33831], docs)

NameError: name 'most_similar' is not defined

In [23]:
verses = scriptures.select("scripture_text").to_series().to_list()[:10]
verses = list(nlp.pipe(verses))
for i in range(len(verses)):
    verses[i] = ' '.join([token.text for token in verses[i] if not token.is_stop and not token.is_punct])
    
verses = list(nlp.pipe(verses))
verses

[beginning God created heaven earth,
 earth form void darkness face deep Spirit God moved face waters,
 God said Let light light,
 God saw light good God divided light darkness,
 God called light Day darkness called Night evening morning day,
 God said Let firmament midst waters let divide waters waters,
 God firmament divided waters firmament waters firmament,
 God called firmament Heaven evening morning second day,
 God said Let waters heaven gathered unto place let dry land appear,
 God called dry land Earth gathering waters called Seas God saw good]

In [80]:
# Compute cosine similarity
search_verse = verses[10]
# for i, doc in enumerate(verses):
#     print(i, search_verse.similarity(doc))

def most_similar(search_verse, verses, n=5):
    similarities = [search_verse.similarity(doc) for doc in verses]
    return sorted(enumerate(similarities), key=lambda item: -item[1])[:n]

related = most_similar(search_verse, verses)
related = pl.DataFrame(related, 
                       schema={"verse": pl.UInt32, "similarity": pl.Float64},
                       orient="row")
related

verse,similarity
u32,f64
10,1.0
41412,0.979598
11,0.978849
41810,0.970631
41413,0.956646


In [81]:
scriptures.with_row_count().join(related, left_on="row_nr", right_on="verse")

row_nr,volume_title,book_title,book_short_title,chapter_number,verse_number,verse_title,verse_short_title,scripture_text,similarity
u32,str,str,str,i64,i64,str,str,str,f64
10,"""Old Testament""","""Genesis""","""Gen.""",1,11,"""Genesis 1:11""","""Gen. 1:11""","""And God said, …",1.0
11,"""Old Testament""","""Genesis""","""Gen.""",1,12,"""Genesis 1:12""","""Gen. 1:12""","""And the earth …",0.978849
41412,"""Pearl of Great…","""Moses""","""Moses""",2,11,"""Moses 2:11""","""Moses 2:11""","""And I, God, sa…",0.979598
41413,"""Pearl of Great…","""Moses""","""Moses""",2,12,"""Moses 2:12""","""Moses 2:12""","""And the earth …",0.956646
41810,"""Pearl of Great…","""Abraham""","""Abr.""",4,11,"""Abraham 4:11""","""Abr. 4:11""","""And the Gods s…",0.970631


In [12]:
x = nlp("For God so loved the world, that he gave his only begotten Son, that whosoever believeth in him should not perish, but have everlasting life.")
x = ' '.join([token.text for token in x if not token.is_stop and not token.is_punct])
x = nlp(x)
y = nlp("Because that he remembered not to shew mercy, but persecuted the poor and needy man, that he might even slay the broken in heart.")
y = ' '.join([token.text for token in y if not token.is_stop and not token.is_punct])

y = nlp(y)
print(y)
x.similarity(y)

remembered shew mercy persecuted poor needy man slay broken heart


0.786303613283564

In [25]:
print(scriptures.select(pl.col("vec")))[0,0].similarity([0,0,0,0,0,0,0,0,0,0,0,0,0,0,0])

shape: (1, 1)
┌───────────────────────────────────┐
│ vec                               │
│ ---                               │
│ object                            │
╞═══════════════════════════════════╡
│ In the beginning God created the… │
└───────────────────────────────────┘


TypeError: 'NoneType' object is not subscriptable